In [84]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from string import punctuation
import nltk
from nltk.corpus import stopwords
import re
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping  


In [85]:
import warnings
warnings.filterwarnings("ignore")

In [86]:
#функция подсчета f1_score
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [87]:
df = pd.read_csv('data.csv')
df

,text,label
0,&#34;В регионе стало меньше пахнуть Россией&#3...,не по теме
1,&#34;Северному потоку-2&#34; предсказали полуп...,мнения
2,10 лет назад протестующие заняли площадь Тахри...,не по теме
3,130 парков для умных. В регионах открываются в...,не по теме
4,170 лыжников эвакуировали с канатной дороги в ...,не по теме
...,...,...
2151,Япония открестилась от планов размещения морпе...,Международные отношения
2152,Ярославские ученые получили сроки за незаконну...,не по теме
2153,Ярославские ученые получили условные сроки за ...,не по теме
2154,"🧙‍♀ Гороскоп на сегодня, 26 января, для всех з...",не по теме


In [88]:
df['label'].value_counts(normalize=True)

не по теме                 0.413265
политика                   0.165584
COVID-19                   0.115492
Международные отношения    0.082096
мнения                     0.060761
военная тематика           0.051948
аналитика                  0.046846
торговля                   0.033395
меры поддержки             0.014378
проекты                    0.005566
Россия                     0.004638
инвестиция                 0.004174
Социологические опросы     0.001855
Name: label, dtype: float64

In [89]:
df[df.duplicated(['text'])==True]

,text,label


In [90]:
df.iloc[2152].text

'Ярославские ученые получили сроки за незаконную передачу технологий Китаю\nhttps://ria.ru/20210126/uchenye-1594659750.html Ярославские ученые получили сроки за незаконную передачу технологий Китаю Ярославские ученые получили сроки за незаконную передачу технологий Китаю Два бывших сотрудника завода "ОДК-Сатурн" в Рыбинске (Ярославская область) получили условные сроки лишения свободы за незаконный экспорт в Китай технологий,... РИА Новости, 26.01.2021 2021-01-26T16:21 2021-01-26T16:21 2021-01-26T16:21 объединенная двигателестроительная корпорация ярославский областной суд федеральная служба по техническому и экспортному контролю (фстэк россии) китай ярославская область рыбинск происшествия /html/head/meta[@name=\'og:title\']/@content /html/head/meta[@name=\'og:description\']/@content https://cdn24.img.ria.ru/images/153104/45/1531044588_0:0:3282:1847_1920x0_80_0_0_d3f6432d46a7119161d2ac3019478189.jpg ЯРОСЛАВЛЬ, 26 янв – РИА Новости. Два бывших сотрудника завода "ОДК-Сатурн" в Рыбинске (

In [91]:
# df.iloc[2153].text

In [92]:
# df.text.duplicated 

In [93]:
df[df.label == 'аналитика']

,text,label
7,"74% украинцев заметили, что страна катится в п...",аналитика
102,Акции «Абрау-Дюрсо» выросли на 10% на фоне зая...,аналитика
103,Акции Абрау Дюрсо растут на Мосбирже на 10%\nМ...,аналитика
116,"Аналитик считает курс рубля ровным\nМосква, , ...",аналитика
119,Аналитики перечислили популярные туристические...,аналитика
...,...,...
2030,Чего ждать от курса рубля в последнюю неделю я...,аналитика
2036,Через порт Сабетта на Ямале в 2020 году вывезл...,аналитика
2116,Эксперты описали сценарий «цветных революций»\...,аналитика
2120,Эксперты: загрузка российских НПЗ в 2020 году ...,аналитика


In [94]:
# df.iloc[1290].text

In [95]:
df[df.label == 'Социологические опросы']

,text,label
200,Большинство украинцев считают неправильным нап...,Социологические опросы
1290,Партия Зеленского стремительно теряет голоса и...,Социологические опросы
1365,Почти три четверти украинцев сочли неправильны...,Социологические опросы
1656,Россияне заявили о готовности пожертвовать сво...,Социологические опросы


In [96]:
df.iloc[1365].text

'Почти три четверти украинцев сочли неправильным выбранный властями курс\nПочти три четверти жителей Украины высказали мнение, что их страна движется в неправильном направлении. Об этом свидетельствуют опубликованные во вторник, 26 января, результаты опроса Киевского международного института социологии (КМИС). Так, 74,3% респондентов выбрали отрицательный ответ на вопрос, идут ли дела на Украине в правильном направлении. Лишь 10% считают, что дела движутся в правильном направлении, а 15,7% респондентов не смогли определиться или отказались отвечать на вопрос. Наименьшее число участников опроса верит в правильность курса на востоке страны (6,6%) и в Донбассе (4,6%). Опрос проводился методом телефонных интервью среди 1005 респондентов в возрасте 18 лет и старше. 21 января бывший министр экономики Украины Виктор Суслов заявил, что конфликт в Донбассе можно решить прямыми переговорами Киева с самопровозглашенными Донецкой и Луганской народными республиками. Министр иностранных дел России С

In [97]:
stopwordslist = set(get_stop_words("russian")) #+ ['это', 'которые']
lemmatizer = WordNetLemmatizer()
ptrn_url = r'\w+:\/+[a-zA-Z0-9\.\/\_\:\-]+'
ptrn_meta = r'(\/.\w+){1,3}'
ptrn_meta2 = r"\[@[\w]+\=(.){1,2}\w+\:\w+\\']"
ptrn_data = r'(\d+\.){2}\d{4}|(\d+\-\d+){2}\d+\w\d+:\d+'
ptrn_phone = r'\d[\ \d{3}]+\-\d+'
# ptrn = r'[^a-zA-Zа-яА-Я0-9\%]'
ptrn = r'[^а-яА-Я0-9\%]'

def remove_url(text):
    text = re.sub(ptrn_url, '', text)
    text = re.sub(ptrn_meta, '', text)
    text = re.sub(ptrn_meta2, '', text)
    text = re.sub(ptrn_data, '', text)
    text = re.sub(ptrn_phone, '', text)
    return text

def words_only(text):
    text=str(text)
    return text.lower()

def remove_punkt(text):
    return re.sub(ptrn, ' ', text)

def to_token(text):
    return nltk.tokenize.word_tokenize(text)

def remove_stopwords(text):
    text_list = [w for w in text if w not in stopwordslist]
    return ' '.join(word for word in text_list)

def morphe_text(text):
    text = [lemmatizer.lemmatize(word) for word in text.split() if word not in stopwordslist]
    return " ".join(text)

def normalize_text(text):
    text = remove_url(text)
    text = words_only(text)
    text = remove_punkt(text)
    text = to_token(text)
    text = remove_stopwords(text)
    text = morphe_text(text)
    return text

In [98]:
df['Content_norm'] = df['text'].apply(normalize_text)

In [99]:
df['Content_norm']

0       34 регионе стало пахнуть россией 34 главные пр...
1       34 северному потоку 2 34 предсказали полупусто...
2       10 протестующие заняли площадь тахрир каире от...
3       130 парков умных регионах открываются новые кв...
4       170 лыжников эвакуировали канатной дороги крас...
                              ...                        
2151    япония открестилась планов размещения морпехов...
2152    ярославские ученые получили сроки незаконную п...
2153    ярославские ученые получили условные сроки пер...
2154    гороскоп 26 января знаков зодиака ежедневный г...
2155    гороскоп 26 января знаков зодиака ежедневный г...
Name: Content_norm, Length: 2156, dtype: object

In [100]:
df.text[2152]

'Ярославские ученые получили сроки за незаконную передачу технологий Китаю\nhttps://ria.ru/20210126/uchenye-1594659750.html Ярославские ученые получили сроки за незаконную передачу технологий Китаю Ярославские ученые получили сроки за незаконную передачу технологий Китаю Два бывших сотрудника завода "ОДК-Сатурн" в Рыбинске (Ярославская область) получили условные сроки лишения свободы за незаконный экспорт в Китай технологий,... РИА Новости, 26.01.2021 2021-01-26T16:21 2021-01-26T16:21 2021-01-26T16:21 объединенная двигателестроительная корпорация ярославский областной суд федеральная служба по техническому и экспортному контролю (фстэк россии) китай ярославская область рыбинск происшествия /html/head/meta[@name=\'og:title\']/@content /html/head/meta[@name=\'og:description\']/@content https://cdn24.img.ria.ru/images/153104/45/1531044588_0:0:3282:1847_1920x0_80_0_0_d3f6432d46a7119161d2ac3019478189.jpg ЯРОСЛАВЛЬ, 26 янв – РИА Новости. Два бывших сотрудника завода "ОДК-Сатурн" в Рыбинске (

In [101]:
df.Content_norm[2152] 

'ярославские ученые получили сроки незаконную передачу технологий китаю ярославские ученые получили сроки незаконную передачу технологий китаю ярославские ученые получили сроки незаконную передачу технологий китаю бывших сотрудника завода одк сатурн рыбинске ярославская область получили условные сроки лишения свободы незаконный экспорт китай технологий риа новости объединенная двигателестроительная корпорация ярославский областной суд федеральная служба техническому экспортному контролю фстэк россии китай ярославская область рыбинск происшествия ярославль 26 янв риа новости бывших сотрудника завода одк сатурн рыбинске ярославская область получили условные сроки лишения свободы незаконный экспорт китай технологий использованы числе создании оружия сообщает ярославский областной суд ученый виноградов приговорен 4 годам лишения свободы условно испытательным сроком 3 штрафом 800 рублей коллега красильников получил 3 условно испытательным сроком 2 6 месяцев суд выяснил апреле 2018 виноградо

In [102]:
df.Content_norm[2155]

'гороскоп 26 января знаков зодиака ежедневный гороскоп 5 26 января растущая луна знаке рак одарит энергией внутренними силами новых свершений благоприятный старта новых проектов смело продвигайте предложения обязательно найдут отклик приняты существует риск поражения начинании поэтому планированию подготовке следует уделить пристальное внимание хорош укрепления родственных отношений овен овны умеют мыслить глобально мечтать больших вещах строить смелые планы доходит дела теряют смелость боятся сделать шаг проявить полной мере талантливы сильны сомневайтесь своем потенциале стоит страшиться успеха иначе ускользнуть действуйте результат заставит ждать космический совет утвердитесь мысли рождены побед телец беспокойство помогало делах тельцы королями действительности переживания страхи столько помогают мешают вашему успеху позволяйте эмоциям брать верх возьмите руки делайте возможное достижения цели приложите максимум усилий вселенная заботится подозреваете сотрудничайте будьте смелыми ре

In [103]:
dubl = df.duplicated(['Content_norm'])
df[dubl == True]

,text,label,Content_norm
1343,"Посол Индии в России рассказал, как себя чувст...",COVID-19,посол индии россии рассказал чувствует прививк...
1781,"Слуцкий заявил, что добиться выхода России из ...",мнения,слуцкий заявил добиться выхода россии пасе пол...
1802,"Соратник Навального рассказал, как его вывезли...",не по теме,соратник навального рассказал вывезли границу ...
2155,"🧙‍♀ Гороскоп на сегодня, 26 января, для всех з...",не по теме,гороскоп 26 января знаков зодиака ежедневный г...


In [104]:
df = df[dubl == False]
df.sample(3)

,text,label,Content_norm
267,В Индонезии за незаконную ловлю рыбы задержали...,Международные отношения,индонезии незаконную ловлю рыбы задержали судн...
2038,Чернышенко возглавит оргкомитет Специальной Ол...,не по теме,чернышенко возглавит оргкомитет специальной ол...
1181,На субсидии для производителей сельхозтехники ...,меры поддержки,субсидии производителей сельхозтехники выделил...


In [105]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42, stratify=df['label'])
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((1441, 3), (711, 3))

In [106]:
df_train['label'].value_counts()

не по теме                 595
политика                   239
COVID-19                   166
Международные отношения    118
мнения                      87
военная тематика            75
аналитика                   68
торговля                    48
меры поддержки              21
проекты                      8
Россия                       7
инвестиция                   6
Социологические опросы       3
Name: label, dtype: int64

Готовим словарь

In [107]:
train_corpus = " ".join(df_train["Content_norm"])
train_corpus[:100]

'сбр ожидает ответа символике чемпионате следующей неделе тасс 24 января международный союз биатлонис'

In [108]:
tokens = nltk.tokenize.word_tokenize(train_corpus)
tokens[:10]

['сбр',
 'ожидает',
 'ответа',
 'символике',
 'чемпионате',
 'следующей',
 'неделе',
 'тасс',
 '24',
 'января']

In [109]:
tokens_filtered = [word for word in tokens if word.isalnum()] 

dist = FreqDist(tokens_filtered)
dist

FreqDist({'января': 5994, 'россии': 5754, '2021': 3993, 'сша': 3450, 'текст': 3152, 'заявил': 2880, 'ранее': 2726, 'новости': 2209, 'фото': 1919, 'акции': 1913, ...})

In [110]:
max_words = 2000
max_len = 100
num_classes = len(df['label'].unique())

In [111]:
train_corpus = " ".join(df['Content_norm'])
train_tokens = nltk.tokenize.word_tokenize(train_corpus)
train_corpus[:100]


'34 регионе стало пахнуть россией 34 главные претензии пашиняну армении продолжаются акции протеста о'

In [112]:

train_tokens_filtered = [word for word in train_tokens if word.isalnum()]
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]


[('января', 9207),
 ('россии', 8809),
 ('2021', 6188),
 ('сша', 5127),
 ('текст', 4910),
 ('заявил', 4396),
 ('ранее', 4163),
 ('новости', 3444),
 ('коронавируса', 3023),
 ('фото', 2984)]

In [113]:
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'января': 1,
 'россии': 2,
 '2021': 3,
 'сша': 4,
 'текст': 5,
 'заявил': 6,
 'ранее': 7,
 'новости': 8,
 'коронавируса': 9,
 'фото': 10,
 'акции': 11,
 'путин': 12,
 'передает': 13,
 'президента': 14,
 '25': 15,
 'страны': 16,
 'риа': 17,
 'москве': 18,
 'навального': 19,
 'президент': 20,
 'словам': 21,
 'глава': 22,
 'отметил': 23,
 '1': 24,
 '2': 25,
 '26': 26,
 '2020': 27,
 'украины': 28,
 '5': 29,
 'вакцины': 30,
 'россия': 31,
 'российской': 32,
 '19': 33,
 'пресс': 34,
 '23': 35,
 'тыс': 36,
 'песков': 37,
 'тасс': 38,
 'людей': 39,
 'владимир': 40,
 'россию': 41,
 'россией': 42,
 'якобы': 43,
 'сообщил': 44,
 'назвал': 45,
 'взгляд': 46,
 '3': 47,
 'рф': 48,
 '4': 49,
 'путина': 50,
 'является': 51,
 'напомним': 52,
 '10': 53,
 'дмитрий': 54,
 '24': 55,
 'связи': 56,
 'вопрос': 57,
 'данным': 58,
 'стороны': 59,
 'украине': 60,
 'мид': 61,
 'алексея': 62,
 'российского': 63,
 'власти': 64,
 'спутник': 65,
 'российских': 66,
 'граждан': 67,
 'компании': 68,
 'поэтому': 69,
 'с

In [114]:
tokenizer = Tokenizer(num_words=max_words, filters='!', char_level=False, split=" ")
tokenizer.fit_on_texts(df_train["Content_norm"])

In [115]:
textsIndexes = tokenizer.texts_to_sequences(df_train["Content_norm"])
x_train = pad_sequences(textsIndexes, maxlen=max_len)

In [116]:
x_train

array([[   0,    0,    0, ...,   59,  157,  238],
       [ 605,  376, 1297, ...,  390,  790,  440],
       [  57,   77,  410, ...,  466,  121,  301],
       ...,
       [ 352,   27,  152, ...,  207,    9, 1808],
       [  18,    1,  360, ...,  289,   32,   36],
       [   0,  292, 1637, ..., 1419,  139,    2]])

In [117]:
textsIndexes_test = tokenizer.texts_to_sequences(df_test["Content_norm"])
x_test = pad_sequences(textsIndexes_test, maxlen=max_len)

In [118]:
x_test

array([[ 762,  586,    3, ...,  549, 1476, 1781],
       [   4,  360,  176, ...,    2,  167,   50],
       [  42,   15,   41, ...,   14,   53,  711],
       ...,
       [   0,    0,    0, ...,  234,   30,  180],
       [   0,    0,    0, ..., 1949,  865,  580],
       [  15,   19,    8, ..., 1839,  487, 1326]])

In [119]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['label']) 
test_enc_labels = le.transform(df_test['label'])
le.classes_

array(['COVID-19', 'Международные отношения', 'Россия',
       'Социологические опросы', 'аналитика', 'военная тематика',
       'инвестиция', 'меры поддержки', 'мнения', 'не по теме', 'политика',
       'проекты', 'торговля'], dtype=object)

In [120]:
train_enc_labels


array([ 9,  4, 10, ...,  4,  0,  9])

In [121]:
num_classes = len(df['label'].unique())
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [122]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=1024, input_length=max_len)) 
model.add(Conv1D(1024, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(516))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [123]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [124]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
# early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/20
3/3 [==============================] - 1s 219ms/step - loss: 2.2526 - get_f1: 0.0000e+00 - val_loss: 1.8098 - val_get_f1: 0.4397
Epoch 2/20
3/3 [==============================] - 0s 154ms/step - loss: 1.8436 - get_f1: 0.3692 - val_loss: 1.7281 - val_get_f1: 0.0658
Epoch 3/20
3/3 [==============================] - 0s 153ms/step - loss: 1.6733 - get_f1: 0.0674 - val_loss: 1.6973 - val_get_f1: 0.0533
Epoch 4/20
3/3 [==============================] - 0s 153ms/step - loss: 1.5252 - get_f1: 0.3219 - val_loss: 1.5928 - val_get_f1: 0.3789
Epoch 5/20
3/3 [==============================] - 0s 154ms/step - loss: 1.3657 - get_f1: 0.5354 - val_loss: 1.4448 - val_get_f1: 0.4926
Epoch 6/20
3/3 [==============================] - 0s 153ms/step - loss: 1.2024 - get_f1: 0.6221 - val_loss: 1.3661 - val_get_f1: 0.6271
Epoch 7/20
3/3 [==============================] - 0s 152ms/step - loss: 1.0380 - get_f1: 0.6629 - val_loss: 1.3562 - val_get_f1: 0.6102
Epoch 8/20
3/3 [============================

In [125]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

2/2 [==============================] - 0s 23ms/step - loss: 1.4998 - get_f1: 0.6182


Test loss: 1.499793291091919
Test f1_score: 0.6182106137275696


In [126]:
preds = model.predict(x_test)
print(classification_report(test_enc_labels, preds.argmax(axis=1), target_names=le.classes_))

23/23 [==============================] - 0s 5ms/step
                         precision    recall  f1-score   support

               COVID-19       0.75      0.72      0.73        82
Международные отношения       0.48      0.36      0.41        59
                 Россия       0.00      0.00      0.00         3
 Социологические опросы       0.00      0.00      0.00         1
              аналитика       0.42      0.39      0.41        33
       военная тематика       0.68      0.35      0.46        37
             инвестиция       0.00      0.00      0.00         3
         меры поддержки       0.25      0.10      0.14        10
                 мнения       0.29      0.19      0.23        43
             не по теме       0.67      0.89      0.76       294
               политика       0.67      0.55      0.60       118
                проекты       0.00      0.00      0.00         4
               торговля       0.47      0.33      0.39        24

               accuracy            

In [127]:
from tensorflow.keras.layers import Conv1D, GlobalMaxPool1D, MaxPooling1D
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Flatten

In [128]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=1024, input_length=max_len))
model.add(Conv1D(filters=516, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128))
model.add(Dropout(0.1))
model.add(Dense(56, activation="relu"))
model.add(Dense(num_classes, activation='softmax'))

In [129]:
model.compile(loss='categorical_crossentropy', #y_pred должен быть распределением вероятностей, y_true -- one-hot кодированный тензор
              optimizer='adam',
              metrics=[get_f1])

In [130]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
# early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    # callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/20
3/3 [==============================] - 2s 291ms/step - loss: 2.4828 - get_f1: 0.0000e+00 - val_loss: 2.2071 - val_get_f1: 0.0000e+00
Epoch 2/20
3/3 [==============================] - 0s 108ms/step - loss: 2.1367 - get_f1: 0.0424 - val_loss: 1.9245 - val_get_f1: 0.4429
Epoch 3/20
3/3 [==============================] - 0s 108ms/step - loss: 1.9921 - get_f1: 0.3258 - val_loss: 1.8205 - val_get_f1: 0.0000e+00
Epoch 4/20
3/3 [==============================] - 0s 108ms/step - loss: 1.8672 - get_f1: 0.0000e+00 - val_loss: 1.8093 - val_get_f1: 0.0000e+00
Epoch 5/20
3/3 [==============================] - 0s 108ms/step - loss: 1.7833 - get_f1: 0.1504 - val_loss: 1.7272 - val_get_f1: 0.4184
Epoch 6/20
3/3 [==============================] - 0s 107ms/step - loss: 1.6872 - get_f1: 0.4705 - val_loss: 1.6615 - val_get_f1: 0.4390
Epoch 7/20
3/3 [==============================] - 0s 107ms/step - loss: 1.5569 - get_f1: 0.4810 - val_loss: 1.6142 - val_get_f1: 0.3696
Epoch 8/20
3/3 [============

In [131]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

2/2 [==============================] - 0s 20ms/step - loss: 2.2072 - get_f1: 0.5319


Test loss: 2.2072198390960693
Test f1_score: 0.531880795955658


In [132]:
preds = model.predict(x_test)

23/23 [==============================] - 0s 6ms/step


In [133]:
print(classification_report(test_enc_labels, preds.argmax(axis=1), target_names=le.classes_))

                         precision    recall  f1-score   support

               COVID-19       0.75      0.67      0.71        82
Международные отношения       0.28      0.31      0.29        59
                 Россия       0.00      0.00      0.00         3
 Социологические опросы       0.00      0.00      0.00         1
              аналитика       0.13      0.12      0.12        33
       военная тематика       0.55      0.32      0.41        37
             инвестиция       0.00      0.00      0.00         3
         меры поддержки       0.00      0.00      0.00        10
                 мнения       0.23      0.14      0.17        43
             не по теме       0.63      0.78      0.70       294
               политика       0.48      0.43      0.45       118
                проекты       0.00      0.00      0.00         4
               торговля       0.24      0.21      0.22        24

               accuracy                           0.53       711
              macro avg